# Análisis de Cobertura de FastText

En este notebook se analiza la cobertura del vocabulario del corpus de noticias respecto al modelo preentrenado FastText. Se evaluará qué porcentaje de palabras del corpus están presentes en FastText.

Importar todas las librerías necesarias

In [1]:
import fasttext.util
import pandas as pd
import os
from collections import Counter
from nltk.tokenize import word_tokenize
import nltk

Descargar recursos necesarios de NLTK

In [2]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /home/adiez/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

Definir directorios de donde queremos cargar los datos resumidos y la ubicación del modelo FastText

In [3]:
# Usa el directorio actual del notebook como raíz del proyecto
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Define las rutas a los directorios de datos
SUMMARIES_DIR = os.path.join(ROOT_DIR, 'data', 'news', 'summarized')
FASTTEXT_PATH = os.path.join(ROOT_DIR, 'models', 'cc.en.300.bin')

print(f"Directorio de resúmenes: {SUMMARIES_DIR}")
print(f"Ruta de FastText: {FASTTEXT_PATH}")

Directorio de resúmenes: /home/adiez/Desktop/News2Stock/data/news/summarized
Ruta de FastText: /home/adiez/Desktop/News2Stock/models/cc.en.300.bin


Opcionalmente descargar el modelo FastText si no está disponible

In [4]:
DOWNLOAD = False

if DOWNLOAD:
    fasttext.util.download_model('en', if_exists='ignore')  # Inglés

Cargar modelo preentrenado de FastText

In [5]:
ft = fasttext.load_model(FASTTEXT_PATH)
print("Modelo FastText cargado.")

Modelo FastText cargado.


Cargar dataset de ejemplo para análisis

In [6]:
# Cargar dataset
df = pd.read_parquet(os.path.join(SUMMARIES_DIR, 'nvidia_news.parquet'), engine="pyarrow")

# Concatenar título y resumen
df["text"] = df["title"] + "\n\n" + df["body_summary"]
df = df.drop(columns=["clean_body", "body_summary", "teaser", "title", "author"])

print(f"Dataset cargado con {len(df)} registros")
print(f"Primeras filas:")
df.head()

Dataset cargado con 10215 registros
Primeras filas:


,id,created,text
0,5120637,"Tue, 06 Jan 2015 10:29:02 -0400",Will NVIDIA's Tegra X1 Dethrone The PlayStatio...
1,5134338,"Fri, 09 Jan 2015 10:35:01 -0400",Barclays: Connected Cars Take Center Stage At ...
2,5192268,"Thu, 29 Jan 2015 13:15:34 -0400",Option Alert: Nvidia Feb $20 Call; 8189 Contra...
3,5220958,"Sun, 08 Feb 2015 11:43:42 -0400","From Coca-Cola To Tesla, Here Are The 68 Most ..."
4,5223302,"Mon, 09 Feb 2015 12:10:15 -0400","Wedbush Previews NVIDIA's Q4 Results, Expects ..."


Tokenizar todo el texto del corpus y construir vocabulario

In [7]:
# Tokenizar todo el texto
all_words = []
for text in df['text']:
    tokens = word_tokenize(text.lower())
    all_words.extend(tokens)

# Contar frecuencias
word_counts = Counter(all_words)
unique_words = set(word_counts.keys())

print(f"Total de palabras (tokens): {len(all_words):,}")
print(f"Palabras únicas: {len(unique_words):,}")

Total de palabras (tokens): 1,019,911
Palabras únicas: 35,127


Obtener vocabulario de FastText

In [8]:
# Obtener vocabulario de FastText
fasttext_vocab = set(ft.get_words())

print(f"Vocabulario de FastText: {len(fasttext_vocab):,} palabras")

Vocabulario de FastText: 2,000,000 palabras



Comparar vocabulario del corpus con FastText y calcular cobertura

In [9]:
# Comparar vocabularios
words_in_fasttext = unique_words & fasttext_vocab
words_not_in_fasttext = unique_words - fasttext_vocab

# Calcular cobertura por tokens (considerando frecuencias)
total_words = len(all_words)
tokens_not_in_fasttext = sum(count for word, count in word_counts.items() if word not in fasttext_vocab)
token_coverage = ((total_words - tokens_not_in_fasttext) / total_words) * 100

# Mostrar resultados
print(f"Total de palabras en corpus: {total_words:,}")
print(f"Palabras únicas en corpus: {len(unique_words):,}")
print(f"Palabras únicas en FastText: {len(words_in_fasttext):,}")
print(f"Palabras únicas NO en FastText: {len(words_not_in_fasttext):,}")
print(f"Cobertura de tokens (por frecuencia): {token_coverage:.2f}%")

Total de palabras en corpus: 1,019,911
Palabras únicas en corpus: 35,127
Palabras únicas en FastText: 23,314
Palabras únicas NO en FastText: 11,813
Cobertura de tokens (por frecuencia): 96.88%
